In [1]:
import numpy as np
import nibabel as nib
import dipy.reconst.dti as dti
import dsiadapt as dsi
import dipy.core.gradients as grad
import matplotlib 
%pylab inline
np.set_printoptions(threshold=numpy.nan)

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Estimate diffusion rates using DTI fitting

fnArr = np.array(['DSI17_exvivo', 'DSI11_invivo_b10k', 'DSI11_invivo_b7k'])
bmaxthrArr = np.array([4500, 2000, 2000]);
adArr = np.zeros(fnArr.shape[0]); # axial diffusion rate

for ii in np.arange(fnArr.shape[0]):
    fn = fnArr[ii];
    data = nib.load('data/' + fn + '_cc.nii.gz').get_data();
    gtab = grad.gradient_table('data/' + fn + '_bvals.txt', 'data/' + fn + '_bvecs.txt');
    mask = gtab.bvals <= bmaxthrArr[ii]; # Use only data with bvals less than 3000
    datamask = data[:, :, :, mask]
    gtabmask = grad.gradient_table_from_bvals_bvecs(gtab.bvals[mask], gtab.bvecs[mask, :])
    disp(gtabmask.bvals.shape) # Number of measurements used for DTI fitting
    disp(gtabmask.bvals.max())
    
    tensormodel = dti.TensorModel(gtabmask)
    tensorfit = tensormodel.fit(datamask)
    adArr[ii] = tensorfit.ad.mean()

adArr = np.concatenate((adArr[0:1], adArr[0:1], adArr)) # Use the ad from ex vivo DSI17 for ex vivo DSI15 and ex vivo DSI11

123
4250.0
57
2000.0
81
1680.0


In [3]:
disp(adArr)

[ 0.00017951  0.00017951  0.00017951  0.00142466  0.00163717]


In [4]:
# Estimate mean diffusion distance
fnArr = np.array(['DSI11_exvivo', 'DSI15_exvivo', 'DSI17_exvivo', 'DSI11_invivo_b10k', 'DSI11_invivo_b7k'])
DELTAArr = np.array([29.4 * 10**-3, 29.4 * 10**-3, 29.4 * 10**-3, 20.9 * 10**-3, 49.2 * 10**-3]); # in second
deltaArr = np.array([16.7 * 10**-3, 16.7 * 10**-3, 16.7 * 10**-3, 12.9 * 10**-3, 42.3 * 10**-3]); # in second
fovArr = np.zeros(fnArr.shape[0]);
mddArr = np.zeros(fnArr.shape[0]);

for ii in np.arange(fnArr.shape[0]):
    fn = fnArr[ii];
    gtab = grad.gradient_table('data/' + fn + '_bvals.txt', 'data/' + fn + '_bvecs.txt');
    DELTA = DELTAArr[ii]; # second
    delta = deltaArr[ii]; # second
    ad = adArr[ii]; # mm2/s
    
    # Compute FOV
    bmax = gtab.bvals.max(); # s/mm2
    qmax = np.sqrt(bmax / (DELTA - delta / 3)) / (2 * np.pi); # mm-1
    deltaq = qmax / dsi.create_qtable(gtab).max(); # mm-1
    fovArr[ii] = 1 / deltaq; # mm
    
    # Compute MDD
    mddArr[ii] = np.sqrt(6 * ad * (DELTA - delta / 3)); # mm
    
    np.savetxt('data/' + fn + '_stats.txt', (mddArr[ii], fovArr[ii]));

In [5]:
disp(mddArr)
disp(fovArr)

[ 0.00506661  0.00506661  0.00506661  0.01191204  0.01856845]
[ 0.02797822  0.0391695   0.04476515  0.04047659  0.07034843]
